# Run

In [7]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import time
import os
from scipy import signal
import librosa
import pickle

In [8]:
with open('Udata.pkl', 'rb') as file:
    Udata = pickle.load(file)

# Vamos remover a atividade 'jumping'
actis = ['climbingdown', 'climbingup', 'lying', 'running', 'sitting', 'standing', 'walking']
posis = ['chest', 'forearm', 'head', 'shin', 'thigh', 'upperarm', 'waist']
users = ['proband' + x for x in np.arange(1,16).astype(str)]
# proband2 não tem acc_climbingup_forearm
users.remove('proband2')
# Vamos remover usuários com menos de 21000 amostras
users.remove('proband1')
users.remove('proband4')
users.remove('proband7')
users.remove('proband14')

In [9]:
def picotar(x, J, passo):
    N = x.shape[0]
    Nj = (N-J)//passo + 1
    X = np.zeros((Nj, J))
    for i in range(Nj):
        X[i,:] = x[(i*passo):(i*passo+J)]
    return X

# Exps iniciais

In [11]:
nu = 0
nd = 0
na = 0
sig = np.array(Udata[nu][nd])[na, :, 1]
px.line(sig)

In [12]:
def lpc(y, m):
    "Return m linear predictive coefficients for sequence y using Levinson-Durbin prediction algorithm"
    #step 1: compute autoregression coefficients R_0, ..., R_m
    R = [y.dot(y)] 
    if R[0] == 0:
        return [1] + [0] * (m-2) + [-1]
    else:
        for i in range(1, m + 1):
            r = y[i:].dot(y[:-i])
            R.append(r)
        R = np.array(R)
    #step 2: 
        A = np.array([1, -R[1] / R[0]])
        E = R[0] + R[1] * A[1]
        for k in range(1, m):
            if (E == 0):
                E = 10e-17
            alpha = - A[:k+1].dot(R[k+1:0:-1]) / E
            A = np.hstack([A,0])
            A = A + alpha * A[::-1]
            E *= (1 - alpha**2)
        return A

In [26]:
def predlin(Xtr):
    X = Xtr[:,:-1]
    y = Xtr[:,-1]
    w = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
    return w

In [13]:
x = sig[500:650]
px.line(x)

In [19]:
a = librosa.lpc(x, order=14)
b = np.hstack([[0], -1 * a[1:]])

In [29]:
p = 14
passo = 1
X = picotar(x, p+1, passo)

In [54]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=a, mode="lines", showlegend=True))
fig.add_trace(go.Scatter(y=b, mode="lines", showlegend=True))
fig.add_trace(go.Scatter(y=A, mode="lines", showlegend=True))
fig.add_trace(go.Scatter(y=predlin(X)[::-1], mode="lines", showlegend=True))
fig.show()

In [51]:
b

array([ 0.        ,  1.77964002, -1.22567732,  0.84223112, -0.62954492,
        0.28421959, -0.17046932,  0.12996257, -0.08625623,  0.06238346,
       -0.10385947,  0.1193972 , -0.09417536,  0.20635475, -0.11755187])

In [50]:
A

array([ 1.        , -1.25695282,  0.21720379, -0.01099859,  0.02637491,
        0.09043737, -0.03796529, -0.08684126,  0.16130687, -0.0367082 ,
       -0.011107  ,  0.03076061, -0.07425119, -0.05144573,  0.05094753])

In [49]:
-predlin(X)

array([ 0.10829653, -0.18762474,  0.08592265, -0.11258415,  0.09595045,
       -0.04880578,  0.05462938, -0.07110867,  0.09342256, -0.22351882,
        0.61799577, -0.88667453,  1.29244591, -1.8150546 ])

In [35]:
xh = X[:,:14].dot(predlin(X))

In [47]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=x[14:], mode="lines", showlegend=True))
fig.add_trace(go.Scatter(y=xh, mode="lines", showlegend=True))
fig.show()

In [ ]:
# Filtro de pré-ênfase e função de janelamento...